<a href="https://colab.research.google.com/github/puja123-std/AI_LABSHEET_PUJA_PURBEY_021-361/blob/main/AGENTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Reflex Based Ai agent
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import random
from IPython.display import HTML

GRID_SIZE    = 7
INTERVAL_MS  = 200
NUM_EPISODES = 3

COLORS = {
    'dirty':   (1.00, 0.39, 0.39),  # Red
    'clean':   (1.00, 1.00, 1.00),  # White
    'visited': (0.39, 0.78, 0.39),  # Green
    'agent':   (0.39, 0.39, 1.00),  # Blue
}

class RandomWalkVacuumAgent:
    def __init__(self, size):
        self.size = size

    def reset(self):
        self.grid = np.random.randint(0, 2, (self.size, self.size))
        self.pos = (random.randrange(self.size), random.randrange(self.size))
        self.visited = {self.pos}

    def neighbors(self, x, y):
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                yield (nx, ny)

    def step(self):
        x, y = self.pos
        if self.grid[x, y] == 0:
            self.grid[x, y] = 1
        else:
            adj = list(self.neighbors(x, y))
            dirty_adj = [p for p in adj if self.grid[p] == 0]
            if dirty_adj:
                nxt = random.choice(dirty_adj)
            else:
                unvisited_adj = [p for p in adj if p not in self.visited]
                if unvisited_adj:
                    nxt = random.choice(unvisited_adj)
                else:
                    nxt = random.choice(adj)
            self.pos = nxt
            self.visited.add(nxt)
        return True

    def run_episode(self):
        frames = []
        total_cells = self.size * self.size
        steps = 0
        while not (np.all(self.grid == 1) and len(self.visited) == total_cells):
            frame = np.zeros((self.size, self.size, 3))
            for i in range(self.size):
                for j in range(self.size):
                    if (i, j) == self.pos:
                        frame[i, j] = COLORS['agent']
                    elif self.grid[i, j] == 0:
                        frame[i, j] = COLORS['dirty']
                    elif (i, j) in self.visited:
                        frame[i, j] = COLORS['visited']
                    else:
                        frame[i, j] = COLORS['clean']
            frames.append(frame)
            self.step()
            steps += 1
        return frames, steps

agent = RandomWalkVacuumAgent(GRID_SIZE)
all_frames = []
total_steps_per_episode = []

for episode_num in range(NUM_EPISODES):
    agent.reset()
    frames, steps = agent.run_episode()
    all_frames.extend(frames)
    total_steps_per_episode.append(steps)
    print(f"Episode {episode_num + 1}: Total steps taken = {steps}")

fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(all_frames[0], interpolation='nearest')

# Add grid lines
ax.set_xticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
ax.set_yticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
ax.grid(which='minor', color='black', linestyle='-', linewidth=1)

ax.set_xticks([])  # Remove major ticks
ax.set_yticks([])
ax.set_xlim(-0.5, GRID_SIZE - 0.5)
ax.set_ylim(GRID_SIZE - 0.5, -0.5)

def update(frame):
    im.set_array(frame)
    return [im]

ani = animation.FuncAnimation(
    fig, update, frames=all_frames,
    interval=INTERVAL_MS, blit=True
)

plt.close(fig)  # Prevent static image output

Episode 1: Total steps taken = 87
Episode 2: Total steps taken = 246
Episode 3: Total steps taken = 198


In [2]:
#Goal Based Ai agent
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import random
from collections import deque
from IPython.display import HTML

GRID_SIZE = 7
INTERVAL_MS = 200

COLORS = {
    'dirty':   (1.00, 0.39, 0.39),  # Red
    'clean':   (1.00, 1.00, 1.00),  # White
    'visited': (0.39, 0.78, 0.39),  # Green
    'agent':   (0.39, 0.39, 1.00),  # Blue
}

class GoalBasedVacuumAgent:
    def __init__(self, size):
        self.size = size

    def reset(self):
        self.grid = np.random.randint(0, 2, (self.size, self.size))  # 0 = dirty, 1 = clean
        self.pos = (random.randint(0, self.size - 1), random.randint(0, self.size - 1))
        self.visited = set()
        self.steps_total = 0
        self.steps_to_clean = 0
        self.goal_reached = False

    def neighbors(self, x, y):
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                yield (nx, ny)

    def bfs_path_to_dirty(self, start):
        queue = deque([(start, [])])
        visited_bfs = set([start])
        while queue:
            (x, y), path = queue.popleft()
            if self.grid[x, y] == 0:
                return path + [(x, y)]
            for nx, ny in self.neighbors(x, y):
                if (nx, ny) not in visited_bfs:
                    visited_bfs.add((nx, ny))
                    queue.append(((nx, ny), path + [(x, y)]))
        return []

    def step(self):
        if np.all(self.grid == 1):
            self.goal_reached = True
            return False

        x, y = self.pos

        if self.grid[x, y] == 0:
            self.grid[x, y] = 1
            self.steps_to_clean += 1

        self.visited.add(self.pos)

        path = self.bfs_path_to_dirty(self.pos)
        if len(path) > 1:
            self.pos = path[1]
        elif len(path) == 1:
            self.pos = path[0]
        else:
            self.goal_reached = True
            return False

        self.steps_total += 1
        return True

    def run_episode(self):
        frames = []
        while True:
            frame = np.zeros((self.size, self.size, 3))
            for i in range(self.size):
                for j in range(self.size):
                    if (i, j) == self.pos:
                        frame[i, j] = COLORS['agent']
                    elif self.grid[i, j] == 0:
                        frame[i, j] = COLORS['dirty']
                    elif (i, j) in self.visited:
                        frame[i, j] = COLORS['visited']
                    else:
                        frame[i, j] = COLORS['clean']
            frames.append(frame)
            if not self.step():
                break
        return frames, self.steps_total, self.steps_to_clean

agent = GoalBasedVacuumAgent(GRID_SIZE)
agent.reset()
frames, steps_total, steps_clean = agent.run_episode()

print(f"✅ Total steps taken: {steps_total}")
print(f"✅ Steps spent cleaning: {steps_clean}")

fig, ax = plt.subplots(figsize=(5,5))
im = ax.imshow(frames[0], interpolation='nearest')
ax.set_xticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
ax.set_yticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlim(-0.5, GRID_SIZE - 0.5)
ax.set_ylim(GRID_SIZE - 0.5, -0.5)

def update(frame):
    im.set_array(frame)
    return [im]

ani = animation.FuncAnimation(
    fig, update, frames=frames,
    interval=INTERVAL_MS, blit=True,
    repeat=False
)
plt.close(fig)

✅ Total steps taken: 35
✅ Steps spent cleaning: 22


In [3]:
#Utility Based Agent

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import random
from collections import deque
from IPython.display import HTML
from matplotlib.patches import Patch

# Simulation parameters
GRID_SIZE    = 7
INTERVAL_MS  = 200
NUM_EPISODES = 3

COLORS = {
    'dirty':   (1.00, 0.00, 0.00),  # Red
    'clean':   (1.00, 1.00, 1.00),  # White
    'visited': (0.00, 1.00, 0.00),  # Green
    'agent':   (0.00, 0.00, 1.00),  # Blue
    'target':  (0.58, 0.00, 0.83),  # Purple
}

class UtilityBasedVacuumAgent:
    def __init__(self, size):
        self.size = size

    def reset(self):
        self.grid    = np.random.randint(0, 2, (self.size, self.size))
        self.pos     = (random.randrange(self.size), random.randrange(self.size))
        self.visited = {self.pos}
        self.target  = None
        self.path    = []
        self.total_steps = 0
        self.cleaning_steps = 0

    def neighbors(self, x, y):
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                yield (nx, ny)

    def find_closest_dirty(self):
        queue = deque([(self.pos, [])])
        seen = {self.pos}
        while queue:
            (x, y), path = queue.popleft()
            if self.grid[x, y] == 0:
                return path
            for nb in self.neighbors(x, y):
                if nb not in seen:
                    seen.add(nb)
                    queue.append((nb, path + [nb]))
        return []

    def step(self):
        x, y = self.pos
        self.total_steps += 1
        if self.grid[x, y] == 0:
            self.grid[x, y] = 1
            self.visited.add((x, y))
            self.cleaning_steps += 1
            self.target = None
            self.path = []
            return True
        if self.path:
            self.pos = self.path.pop(0)
            self.visited.add(self.pos)
            return True
        new_path = self.find_closest_dirty()
        if new_path:
            self.path = new_path
            self.target = new_path[-1]
            return self.step()
        return False

    def run_episode(self):
        frames = []
        while np.any(self.grid == 0):
            canvas = np.zeros((self.size, self.size, 3))
            for i in range(self.size):
                for j in range(self.size):
                    if (i, j) == self.pos:
                        canvas[i, j] = COLORS['agent']
                    elif self.target == (i, j):
                        canvas[i, j] = COLORS['target']
                    elif self.grid[i, j] == 0:
                        canvas[i, j] = COLORS['dirty']
                    elif (i, j) in self.visited:
                        canvas[i, j] = COLORS['visited']
                    else:
                        canvas[i, j] = COLORS['clean']
            frames.append(canvas)
            self.step()
        return frames

agent = UtilityBasedVacuumAgent(GRID_SIZE)
all_frames = []

for episode in range(NUM_EPISODES):
    agent.reset()
    frames = agent.run_episode()
    all_frames.extend(frames)
    print(f"Episode {episode + 1}: Total Steps = {agent.total_steps}, Cleaning Steps = {agent.cleaning_steps}")

fig, ax = plt.subplots(figsize=(5, 5))
fig.subplots_adjust(bottom=0.2)

im = ax.imshow(all_frames[0], interpolation='nearest')
ax.set_xticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
ax.set_yticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlim(-0.5, GRID_SIZE - 0.5)
ax.set_ylim(GRID_SIZE - 0.5, -0.5)

legend_elems = [
    Patch(facecolor=COLORS['dirty'], label='Dirty'),
    Patch(facecolor=COLORS['clean'], label='Clean (not visited)'),
    Patch(facecolor=COLORS['visited'], label='Visited & Clean'),
    Patch(facecolor=COLORS['agent'], label='Agent'),
    Patch(facecolor=COLORS['target'], label='Target'),
]
ax.legend(handles=legend_elems, loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2, frameon=False)

def update(frame):
    im.set_array(frame)
    return [im]

ani = animation.FuncAnimation(fig, update, frames=all_frames, interval=INTERVAL_MS, blit=True)
plt.close(fig)

Episode 1: Total Steps = 84, Cleaning Steps = 24
Episode 2: Total Steps = 61, Cleaning Steps = 18
Episode 3: Total Steps = 88, Cleaning Steps = 27
